#### Name : Kaustubh Agawane
#### Student ID : 21260460
#### Email : kaustubh.agawane2@mail.dcu.ie

#### Assignment 1: Data Analysis
#### Task 4 : Calculate TF-IDF with MapReduce/Pig/Hive


#### 1) Package Imports

In [1]:
from pyhive import hive
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

#### 2) Hive Connection setup

In [2]:
host="127.0.0.1"
port=10000
username="kaustubh_agawane2"
password="5567170637024851622"
conn = hive.Connection(host, port, username, password)

#### 3) Finding top 10 users to calculate the per-user TF-IDF

In [3]:
data = pd.read_sql("select OwnerUserId, max(DisplayName) as Displayname, sum(score) as score from stackexposts group by OwnerUserId order by score desc limit 10", conn)
display(data)

,owneruserid,displayname,score
0,87234,GManNickG,37672
1,4883,readonly,28817
2,9951,e-satis,26878
3,6068,pupeno,25944
4,89904,Hamza Yerlikaya,24024
5,51816,Joan Venge,23763
6,49153,Ali,20203
7,179736,TIMEX,19603
8,95592,Matthew Rankin,19479
9,63051,flybywire,19362


#### 4) Selecting Post Body and Titles for each of top 10 users

In [4]:
owner_user_id=list(data['owneruserid'])
query=f"select owneruserid,displayname,body,title,score from stackexposts where owneruserid in {tuple(owner_user_id)}" 
df = pd.read_sql(query, conn)
df['Body-Title']=df['body'] + ' ' + df['title']

#### 5) Function to calculate TF-IDF on post Body and Title fields and finding top 10 terms for each of the top 10 user

##### Steps performed in below function
###### Step 1 : Passing Title dataframe
###### Step 2 & Step 3 : TfidfVectorizer.fit_transform() function filters the stop words, cleans the data for unwanted characters and calculates TF-IDF on entire list of Title dataframe for each term.
###### Step 4 : Creating a new dataframe after calculating TF-IDF
###### Step 5 : To find the top 10 terms, we use dataframe sum() function to sum TF-IDF for every term and using nlargest() function we find top 10 terms.

In [5]:
def calc_tf_idf(pass_df):
    df_tfidf=pass_df
    vectorizer = TfidfVectorizer(stop_words='english', lowercase=True)
    stop_words=vectorizer.fit_transform(df_tfidf) #Passing Title Dataframe to calculate TF-IDF
    df_calculated_tfidf = pd.DataFrame(stop_words.toarray(),columns=vectorizer.get_feature_names())
    sum_of_tf_idf=df_calculated_tfidf.sum(axis = 0)
    calc_top_10=sum_of_tf_idf.nlargest(10)
    find_top10_words = list(calc_top_10.index)
    return df_calculated_tfidf[find_top10_words]

#### 6) Displaying Top 10 terms TF-IDF for each of top 10 users

In [6]:
for i in data.index:
        extact_namer=data['displayname'].values[i]
        df1=df.query(f"displayname == '{extact_namer}'")
        df2=calc_tf_idf(df1['Body-Title'])
        print("User ID:",extact_namer + "                          Top 10 Words TF-IDF")
        print("-------------------------------------------------------------------------")
        df2.insert(0,'displayname',extact_namer)
        display(df2)

User ID: GManNickG                          Top 10 Words TF-IDF
-------------------------------------------------------------------------


,displayname,lt,copy,data,int,idiom,gt,quot,swap,arraysize,array
0,GManNickG,0.431818,0.000000,0.51101,0.172727,0.000000,0.129546,0.086364,0.000000,0.283895,0.194318
1,GManNickG,0.126037,0.000000,0.00000,0.252073,0.000000,0.252073,0.252073,0.000000,0.000000,0.000000
2,GManNickG,0.000000,0.523691,0.00000,0.000000,0.418953,0.000000,0.000000,0.314215,0.000000,0.079656


User ID: readonly                          Top 10 Words TF-IDF
-------------------------------------------------------------------------


,displayname,python,use,ruby,list,table,branch,way,difference,git,process
0,readonly,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.350231
1,readonly,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.203342,0.000000
2,readonly,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.149441,0.000000
3,readonly,0.000000,0.000000,0.413988,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,readonly,0.000000,0.000000,0.000000,0.000000,0.000000,0.636117,0.000000,0.000000,0.202994,0.000000
5,readonly,0.152171,0.174938,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,readonly,0.000000,0.191672,0.000000,0.000000,0.000000,0.000000,0.000000,0.491935,0.000000,0.000000
7,readonly,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,readonly,0.000000,0.000000,0.204627,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,readonly,0.251336,0.000000,0.000000,0.000000,0.000000,0.000000,0.144470,0.000000,0.000000,0.000000


User ID: e-satis                          Top 10 Words TF-IDF
-------------------------------------------------------------------------


...

,displayname,python,want,user,string,gt,dictionary,lt,use,javascript,return
0,TIMEX,0.285104,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
1,TIMEX,0.134952,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
2,TIMEX,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.124939,0.056344,0.000000,0.000000
3,TIMEX,0.000000,0.000000,0.000000,0.177879,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
4,TIMEX,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
110,TIMEX,0.071606,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.099799,0.210055
111,TIMEX,0.053461,0.000000,0.000000,0.000000,0.0,0.07638,0.000000,0.000000,0.000000,0.000000
112,TIMEX,0.000000,0.032024,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
113,TIMEX,0.000000,0.000000,0.132608,0.000000,0.0,0.00000,0.063177,0.000000,0.000000,0.068069


User ID: Matthew Rankin                          Top 10 Words TF-IDF
-------------------------------------------------------------------------


,displayname,install,pip,python,lt,gt,branch,installed,flask,version,django
0,Matthew Rankin,0.000000,0.000000,0.000000,0.000000,0.043561,0.571753,0.000000,0.000000,0.000000,0.000000
1,Matthew Rankin,0.000000,0.000000,0.000000,0.650278,0.544984,0.000000,0.000000,0.000000,0.000000,0.000000
2,Matthew Rankin,0.349045,0.319958,0.319958,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Matthew Rankin,0.235219,0.211697,0.094087,0.000000,0.000000,0.000000,0.109034,0.000000,0.000000,0.054517
4,Matthew Rankin,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.447027,0.533396,0.533396,0.000000
5,Matthew Rankin,0.427251,0.284834,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Matthew Rankin,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.425746
7,Matthew Rankin,0.000000,0.000000,0.338392,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


User ID: flybywire                          Top 10 Words TF-IDF
-------------------------------------------------------------------------


,displayname,file,python,want,java,vs,gt,use,bash,instance,standard
0,flybywire,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,flybywire,0.000000,0.000000,0.112601,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,flybywire,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,flybywire,0.000000,0.120377,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,flybywire,0.399837,0.000000,0.000000,0.000000,0.000000,0.300322,0.157240,0.165412,0.000000,0.498416
5,flybywire,0.166264,0.000000,0.155217,0.000000,0.000000,0.000000,0.000000,0.412698,0.000000,0.466327
6,flybywire,0.000000,0.000000,0.000000,0.657184,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,flybywire,0.099270,0.111844,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,flybywire,0.000000,0.461401,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,flybywire,0.000000,0.000000,0.123087,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
